In [17]:
import pandas as pd
from typing import *
import re
from fpgrowth_py import fpgrowth

In [5]:
files = pd.read_csv(r"../Generated_Result/three_output1.csv", sep=";").fillna("Yes")
obj = files.iloc[:, :3]
sets = obj.values.tolist()

In [6]:
for idx, subset in enumerate(sets):
    subset = [val.split("|") for val in subset]
    subset = [item.strip() for yitem in subset for item in yitem]
    sets[idx] = subset

In [7]:
def myFunc(val: List[str]):
    pattern = r'\b(?:yes|Yes)\b'
    processed_list = [item for item in val if not re.search(pattern, item)]
    return processed_list

In [14]:
# sets_proper = filter(myFunc, sets)
# sets_proper=list(sets_proper)
sets_proper = [myFunc(val) for val in sets]

In [83]:
freqitemset, rules = fpgrowth(sets_proper, minSupRatio=0.0012, minConf=0.6)

In [84]:
finals = pd.DataFrame(rules, columns=["Start", "to", "confidence ratio"])
finals["Start"] = finals["Start"].apply(lambda x: next(iter(x)))
finals["to"] = finals["to"].apply(lambda x: next(iter(x)))

In [61]:
# finals[["Start", "to"]] = finals[["Start", "to"]].apply(lambda x: next(iter(x)))

In [87]:
# finals[finals.to.apply(lambda x: len(x.split(","))>1)]
finals.shape

(584653, 3)

In [73]:
sets_proper

[['Electricity generation sector',
  'Increased emissions through offsets',
  'Host countries',
  'project developers',
  'investors'],
 ['Energy infrastructure projects (EU support for 16 natural gas projects)',
  'Limited lifetime of fossil investments due to decarbonization agenda',
  'Project sponsors',
  'implementers',
  'and regulatory bodies involved in the energy sector'],
 ['Construction project',
  'Delayed project completion',
  'Homeowners',
  'builders',
  'architects'],
 ['Electricity sector',
  'Emission reduction opportunities',
  'Project developers',
  'regulators',
  'environment agencies'],
 ['Road construction',
  'Delay',
  'cost overruns',
  'instability in the industry',
  'Construction companies',
  'government agencies',
  'affected communities'],
 ['Overland natural gas pipeline connecting India and Iran via Pakistan.',
  'Political tensions between India',
  'Iran',
  'and Pakistan.',
  'India',
  'Iran',
  'Pakistan',
  'and private companies involved in t